# Voice Agent for Conversational AI with Pipecat
In this notebook, we walk through how to craft and deploy a voice AI bot using Pipecat AI. We illustrate the basic Pipecat flow with the `meta/llama-3.3-70b-instruct`* LLM model and Riva for STT (Speech-To-Text) & TTS (Text-To-Speech). However, Pipecat is not opinionated and other models and STT/TTS services can easily be used. See [Pipecat documentation](https://docs.pipecat.ai/server/services/supported-services#supported-services) for other supported services.

Pipecat AI is an open-source framework for building voice and multimodal conversational agents. Pipecat simplifies the complex voice-to-voice AI pipeline, and lets developers build AI capabilities easily and with Open Source, commercial, and custom models. See [Pipecat's Core Concepts](https://docs.pipecat.ai/getting-started/core-concepts) for a deep dive into how it works.

The framework was developed by Daily, a company that has provided real-time video and audio communication infrastructure since 2016. It is fully vendor neutral and is not tightly coupled to Daily's infrastructure. That said, we do use it in this demo. Sign up for a Daily-bot API key [here](https://bots.daily.co/sign-up).

> [Development Note]: *We are using "meta/llama-3.3-70b-instruct" for now because it works with tool calling, but can update/change this model at any time. It is a one line change in the notebook.

## Step 1 - Install dependencies
First we set our environment.

We use Daily for transport, OpenAI for context aggregation, Riva for TTS & TTS, and Silero for VAD (Voice Activity Detection). If using different services, for example Cartesia for TTS, one would run `pip install pipecat-ai[cartesia]`.

> [Development note]: We're installing from the github main branch here to ensure we have the latest improvements. By the time we address feedback we'll have a new release of Pipecat and just install the pipecat parts we are using.

In [ ]:
!pip install python-dotenv
%load_ext dotenv
%dotenv

!pip install "git+https://github.com/pipecat-ai/pipecat.git@main"
!pip install "pipecat-ai[daily,openai,riva,silero]"

## Step 2 - Configure Daily transport for WebRTC communication
- room_url: Where to connect (and where will navigate to to talk to our bot)
- None: No authentication token needed
- "NVIDIA NIM": The bot's display name
- Enable audio output for text-to-speech playback and enable VAD

In [ ]:
# Url to talk to the NVIDIA NIM bot
# Update to your room url after obtaining Daily-bot API key
#### NOTE: if this is changed, the link in Step 11 markdown will no longer work.
DAILY_SAMPLE_ROOM_URL="https://pc-34b1bdc94a7741719b57b2efb82d658e.daily.co/prod-test"

In [ ]:
from pipecat.audio.vad.silero import SileroVADAnalyzer
from pipecat.transports.services.daily import DailyParams, DailyTransport

transport = DailyTransport(
    DAILY_SAMPLE_ROOM_URL,
    None,
    "NVIDIA NIM Agent",
    DailyParams(
        audio_out_enabled=True,
        vad_enabled=True,
        vad_analyzer=SileroVADAnalyzer(),
        vad_audio_passthrough=True,
    ),
)

## Step 3 - Initialize LLM, STT, and TTS services
We can customize options, for example a different LLM `model` or `voice_id` for FastPitch TTS.

In [ ]:
import os
from pipecat.services.nim import NimLLMService
from pipecat.services.riva import FastPitchTTSService, ParakeetSTTService

stt = ParakeetSTTService(api_key=os.getenv("NVIDIA_API_KEY"))

llm = NimLLMService(
    api_key=os.getenv("NVIDIA_API_KEY"), model="meta/llama-3.3-70b-instruct"
)

tts = FastPitchTTSService(api_key=os.getenv("NVIDIA_API_KEY"))

## Step 4 - Define LLM prompt
Edit the prompt as desired.

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful LLM in a WebRTC call. Your goal is to demonstrate your capabilities in a succinct way. Your output will be converted to audio so don't include special characters in your answers. Respond to what the user said in a creative and helpful way and make a weather pun if it is possible.",
    },
]

## Step 5 - Define tool calling function
Here we use the classic "get_weather" example. We use OpenAI's ChatCompletionToolParam and register the function with the llm.

In [ ]:
from openai.types.chat import ChatCompletionToolParam
from pipecat.frames.frames import TextFrame


async def start_fetch_weather(function_name, llm, context):
    await llm.push_frame(TextFrame("Let me check on that."))
    print(f"Starting fetch_weather_from_api with function_name: {function_name}")

async def fetch_weather_from_api(function_name, tool_call_id, args, llm, context, result_callback):
    await result_callback({"conditions": "nice", "temperature": "75"})

tools = [
            ChatCompletionToolParam(
                type="function",
                function={
                    "name": "get_current_weather",
                    "description": "Returns the current weather at a location, if one is specified, and defaults to the user's location.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The location to find the weather of, or if not provided, it's the default location.",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "Whether to use SI or USCS units (celsius or fahrenheit).",
                            },
                        },
                        "required": ["location", "format"],
                    },
                },
            )
        ]

llm.register_function(None, fetch_weather_from_api, start_callback=start_fetch_weather)

## Step 6 - Initialize the Context Aggregator

In [ ]:
from pipecat.processors.aggregators.openai_llm_context import OpenAILLMContext

context = OpenAILLMContext(messages, tools)
context_aggregator = llm.create_context_aggregator(context)

## Step 7 - Create pipeline
Here we align the services into a pipeline to process speech into text, send to llm, then turn the llm response text into speech.

In [ ]:
from pipecat.pipeline.pipeline import Pipeline

pipeline = Pipeline(
    [
        transport.input(),              # Transport user input
        stt,                            # STT
        context_aggregator.user(),      # User responses
        llm,                            # LLM
        tts,                            # TTS
        transport.output(),             # Transport bot output
        context_aggregator.assistant(), # Assistant spoken responses
    ]
)

## Step 8 - Create PipelineTask

In [ ]:
from pipecat.pipeline.task import PipelineParams, PipelineTask

task = PipelineTask(pipeline, PipelineParams(allow_interruptions=True))

## Step 9 - Create a pipeline runner
This manages the processing pipeline.

In [ ]:
from pipecat.pipeline.runner import PipelineRunner

runner = PipelineRunner()

## Step 10 - Set event handlers
The `on_first_participant_joined` handler tells the bot to start the conversation when you join the call.  
The `on_participant_left` handler sends an EndFrame which signals to terminate the pipeline.

In [ ]:
from pipecat.frames.frames import LLMMessagesFrame, EndFrame

@transport.event_handler("on_first_participant_joined")
async def on_first_participant_joined(transport, participant):
    # Kick off the conversation.
    messages.append({"role": "system", "content": "Please introduce yourself to the user and deliver a weather fact."})
    await task.queue_frames([LLMMessagesFrame(messages)])

@transport.event_handler("on_participant_left")
async def on_participant_left(transport, participant, reason):
    print(f"Participant left: {participant}")
    await task.queue_frame(EndFrame())   

## Step 11 - Run the Agent!

Once you have run the code block below, you can talk to the agent at
#### [https://pc-34b1bdc94a7741719b57b2efb82d658e.daily.co/prod-test](https://pc-34b1bdc94a7741719b57b2efb82d658e.daily.co/prod-test) 
to open a new browser window connected to the agent's WebRTC session.

### Suggested conversations:
- *Try tool calling.* As the bot about the weather.
- *Observe the agent's context "memory".* Ask the agent to recite the alphabet. Interrupt the agent in the middle of the alphabet and ask it another question (What is 256 times 3?). Ask it a few more things... then ask it to pick up where it left off in reciting the alphabet.
- *Play.* Ask the agent to tell a joke. Interrupt it. Ask it to explain the pythagorean theorem.

The first time you run the bot, it will load weights for a voice activity model into the local Python process. This will take 10-15 seconds. A permissions dialog will ask you to allow the browser to access your camera and microphone. Click yes to start talking to the bot. If you have any trouble with this, see [here](https://help.daily.co/en/articles/2525908-allow-camera-and-mic-access).

> [Development Note]: We can build the microphone input and speaker output into the notebook so that it isn't necessary to open another browser window. We will put this on our todo list, along with any feedback we get on Friday from the initial testing of the notebook.

In [ ]:
await runner.run(task)